In [ ]:
#Name : Areeba Nazar
#Assignment no :2
#Assignment topic: Training Of The Model On Given Dataset(iris.csv)
#Submitted to:Nabeegh Ahmad

In [13]:
!pip install torch torchvision pandas numpy matplotlib scikit-learn

In [14]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

In [15]:
# 1. Loading and preprocessing of  the data
data = pd.read_csv('iris.csv') 

X = data.iloc[:, :-1].values  
y = data.iloc[:, -1].values   

# Encoding of  labels to numerical values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Spliting data  into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [16]:
# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convertion to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long) 
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [17]:
# Creation of  DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [18]:
# 2. Defining neural network model
class IrisClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super(IrisClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)  # Output logits (no softmax here, as CrossEntropyLoss handles it)
        return x



In [19]:
# Initialing model, loss, and optimizer
input_size = X_train.shape[1]
num_classes = len(label_encoder.classes_)  
model = IrisClassifier(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [20]:
# 3. Training the model
num_epochs = 50
for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y) 
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/50], Loss: 1.0439
Epoch [2/50], Loss: 1.0001
Epoch [3/50], Loss: 0.9443
Epoch [4/50], Loss: 0.8764
Epoch [5/50], Loss: 0.8723
Epoch [6/50], Loss: 0.8010
Epoch [7/50], Loss: 0.7433
Epoch [8/50], Loss: 0.7401
Epoch [9/50], Loss: 0.5859
Epoch [10/50], Loss: 0.5876
Epoch [11/50], Loss: 0.6001
Epoch [12/50], Loss: 0.4391
Epoch [13/50], Loss: 0.4364
Epoch [14/50], Loss: 0.4123
Epoch [15/50], Loss: 0.4008
Epoch [16/50], Loss: 0.3290
Epoch [17/50], Loss: 0.3907
Epoch [18/50], Loss: 0.2503
Epoch [19/50], Loss: 0.3241
Epoch [20/50], Loss: 0.3503
Epoch [21/50], Loss: 0.2794
Epoch [22/50], Loss: 0.2876
Epoch [23/50], Loss: 0.2740
Epoch [24/50], Loss: 0.3701
Epoch [25/50], Loss: 0.2957
Epoch [26/50], Loss: 0.3483
Epoch [27/50], Loss: 0.2780
Epoch [28/50], Loss: 0.1834
Epoch [29/50], Loss: 0.2478
Epoch [30/50], Loss: 0.2201
Epoch [31/50], Loss: 0.1742
Epoch [32/50], Loss: 0.2032
Epoch [33/50], Loss: 0.1425
Epoch [34/50], Loss: 0.1271
Epoch [35/50], Loss: 0.1371
Epoch [36/50], Loss: 0.1640
E

In [21]:
# 4. Evaluation of accuracy of the model
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_X, batch_y in test_loader:
        outputs = model(batch_X)
        _, predicted = torch.max(outputs.data, 1)
        total += batch_y.size(0)
        correct += (predicted == batch_y).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the model on the test set: {accuracy:.2f}%')

Accuracy of the model on the test set: 100.00%
